In [1]:
import pandas as pd
import lightgbm as lgb
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

print("--- Étape 3 : Entraînement du Modèle d'Assignation de Grade (Le Stratège) ---")

# --- 1. Chargement des données et des assets du Modèle 1 ---
input_path = "../../data/processed/dataset_clean_no_outliers.parquet"
output_dir = "../../models/"
model_1_path = os.path.join(output_dir, "step1_risk_model.pkl")
model_1_cols_path = os.path.join(output_dir, "step1_risk_model_columns.pkl")

df = pd.read_parquet(input_path)
model_1 = joblib.load(model_1_path)
model_1_columns = joblib.load(model_1_cols_path)
print("✅ Données et assets du Modèle 1 chargés.")

# --- 2. Calcul du Score de Risque ---
# Préparation des données pour le Modèle 1
selected_features_for_risk_model = [
    'term', 'annual_inc', 'dti', 'revol_util', 'revol_bal', 'loan_amnt', 
    'emp_length', 'home_ownership', 'purpose', 'verification_status', 
    'mort_acc', 'pub_rec', 'open_acc', 'total_acc'
]
X_for_risk_prediction = df[selected_features_for_risk_model]
X_for_risk_prediction = pd.get_dummies(X_for_risk_prediction, drop_first=True, dtype=float)
X_for_risk_prediction = X_for_risk_prediction.reindex(columns=model_1_columns, fill_value=0)

df['risk_score'] = model_1.predict_proba(X_for_risk_prediction)[:, 1]
print("✅ Score de risque calculé.")

# --- 3. Préparation des données pour le Modèle 3 (Grade) ---
# La Cible (Y) est le 'grade'
y = df['grade']

# Les Features (X) sont le risque, le taux et les infos clés du prêt
# On utilise le taux d'intérêt historique ('int_rate') pour l'entraînement
features_for_grade_model = [
    'risk_score',
    'int_rate',
    'loan_amnt',
    'annual_inc',
    'term'
]
X = df[features_for_grade_model]

# Encodage de la cible (A, B, C... -> 0, 1, 2...)
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# --- 4. Entraînement du Modèle 3 ---
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

grade_model = lgb.LGBMClassifier(objective='multiclass', random_state=42)
print("Entraînement du modèle de classification de grade...")
grade_model.fit(X_train, y_train)
print("✅ Modèle de grade entraîné.")

# --- 5. Évaluation ---
y_pred = grade_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"\n--- Performance du Modèle de Grade ---")
print(f"Accuracy : {accuracy:.4f}")
print("\n--- Rapport de Classification ---")
print(classification_report(y_test, y_pred, target_names=le.classes_))

# --- 6. Sauvegarde ---
grade_model_path = os.path.join(output_dir, "step3_grade_model.pkl")
grade_model_cols_path = os.path.join(output_dir, "step3_grade_model_columns.pkl")
encoder_path = os.path.join(output_dir, "grade_label_encoder.pkl") # On sauvegarde à nouveau pour être sûr

joblib.dump(grade_model, grade_model_path)
joblib.dump(X_train.columns.tolist(), grade_model_cols_path)
joblib.dump(le, encoder_path)

print(f"\n✅ Modèle de grade (Étape 3) et ses assets sauvegardés.")

--- Étape 3 : Entraînement du Modèle d'Assignation de Grade (Le Stratège) ---
✅ Données et assets du Modèle 1 chargés.
✅ Score de risque calculé.
Entraînement du modèle de classification de grade...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001163 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 982
[LightGBM] [Info] Number of data points in the train set: 292971, number of used features: 5
[LightGBM] [Info] Start training from score -1.795710
[LightGBM] [Info] Start training from score -1.200761
[LightGBM] [Info] Start training from score -1.311563
[LightGBM] [Info] Start training from score -1.851106
[LightGBM] [Info] Start training from score -2.556080
[LightGBM] [Info] Start training from score -3.650216
[LightGBM] [Info] Start training from score -5.819336
✅ Modèle de grade entraîné.

--- Performance du Modèle de Grade ---
Acc